<a href="https://colab.research.google.com/github/gulizaaitkulova/AI_Midterm_12204505/blob/main/AI_Midterm_12204505_Guliza_Aitkulova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load Packages
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation


In [ ]:
# Generate Synthetic Data
# The size of the generated data will depend on NumSamples and TimeSteps
def generate_one_sequence(time_steps = 5, fail_prob = 0.1):
  """
    Generate single sequance data with given time steps and failure probability
  """
  # Randomly assign failure value by probability fail_prob
  failure = np.random.choice([0, 1], 1, p=[1 - fail_prob, fail_prob])

  tmp_seq = []
  vbr_seq = []
  spd_seq = []

  # According to the failure generate the synthetic data
  for _ in range(time_steps):
    if failure == 1:
      # Generate distinct data for failure cases
      tmp = np.random.uniform(100, 150)   # Higher temperature for failure
      vbr = np.random.uniform(10, 20)     # Higher vibration for failure
      spd = np.random.uniform(0.5, 1)     # Lower speed for failure
    else:
      # Generate normal data for non-failure cases
      tmp = np.random.uniform(50, 100)    # Normal temperature
      vbr = np.random.uniform(0, 10)      # Normal vibration
      spd = np.random.uniform(1, 5)       # Normal speed

    tmp_seq.append(tmp)
    vbr_seq.append(vbr)
    spd_seq.append(spd)

  tmp_seq = np.array(tmp_seq)
  vbr_seq = np.array(vbr_seq)
  spd_seq = np.array(spd_seq)

  return tmp_seq, vbr_seq, spd_seq, failure


def generate_sequential_data(num_samples=1000, time_steps=5, fail_prob = 0.1):
  """
    Generate synthetic data for training
  """
  temperatures = []
  vibrations = []
  conveyor_speeds = []
  failures = []

  for _ in range(num_samples):
    tmp_seq, vbr_seq, spd_seq, failure = generate_one_sequence(time_steps)
    temperatures.append(tmp_seq)
    vibrations.append(vbr_seq)
    conveyor_speeds.append(spd_seq)
    failures.append(failure)

  temperatures = np.array(temperatures)
  vibrations = np.array(vibrations)
  conveyor_speeds = np.array(conveyor_speeds)
  failures = np.array(failures)

  return temperatures, vibrations, conveyor_speeds, failures

NumSamples = 500
TimeSteps = 5
temperatures, vibrations, conveyor_speeds, failures = generate_sequential_data(NumSamples, TimeSteps)

In [ ]:
# Preprocessing.
# Normalize the using StandardScaler. It will compute mean and variance of the data.
# After that normalization is done by subtracting mean and dividing by variance
from sklearn.preprocessing import StandardScaler

scaler_tmp = StandardScaler()
temperatures = scaler_tmp.fit_transform(temperatures)

scaler_vbr = StandardScaler()
vibrations = scaler_vbr.fit_transform(vibrations)

scaler_spd = StandardScaler()
conveyor_speeds = scaler_spd.fit_transform(conveyor_speeds)

In [ ]:
# Split into Train and test set
# 90% will be train set and 10% test set
split = NumSamples - NumSamples // 10
data = np.dstack((temperatures, vibrations, conveyor_speeds))
x_train = data[:split]
y_train = failures[:split]
x_test = data[split:]
y_test = failures[split:]

In [ ]:
# Build LSTM Model
# The model consists of of single LSTM layer and 3 Dense layers
# Activation functions such as Relu and Sigmoid are used.
# Moreover, Dropout layer is added to prevent overfitting
model = Sequential()
model.add(LSTM(8, input_shape=(5,3), return_sequences=False))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [ ]:
# Compile Model
# MSE loss and Adam optimizer used to train model
model.compile(loss='mse', optimizer ='adam', metrics=['accuracy'])

# Train Model for 25 epochs with batch size 5
model.fit(x_train, y_train, epochs=25, batch_size=5);

Epoch 1/25
90/90 [==============================] - 6s 7ms/step - loss: 0.2400 - accuracy: 0.8644
Epoch 2/25
90/90 [==============================] - 0s 5ms/step - loss: 0.1959 - accuracy: 0.8644
Epoch 3/25
90/90 [==============================] - 0s 5ms/step - loss: 0.1010 - accuracy: 0.8644
Epoch 4/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0701 - accuracy: 0.8644
Epoch 5/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0637 - accuracy: 0.8644
Epoch 6/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0627 - accuracy: 0.8644
Epoch 7/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0599 - accuracy: 0.8644
Epoch 8/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0580 - accuracy: 0.8644
Epoch 9/25
90/90 [==============================] - 0s 5ms/step - loss: 0.0551 - accuracy: 0.8644
Epoch 10/25
90/90 [==============================] - 1s 6ms/step - loss: 0.0535 - accuracy: 0.8644
Epoch 11/25
90/90 [

In [ ]:
# Evaluate Model. Accuracy is used as evaluation metric
scores = model.evaluate(x_test, y_test, verbose=1, batch_size=5)
print('Accurracy: {}'.format(scores[1]))

10/10 [==============================] - 0s 3ms/step - loss: 0.0103 - accuracy: 1.0000
Accurracy: 1.0


In [ ]:
# Real-Time simulation, it will run for 10 iterations
# If failure is detected the program will notify by printing corresponding message
for i in range(10):
  tmp_seq, vbr_seq, spd_seq, failure = generate_one_sequence(time_steps = 5, fail_prob = 0.5)
  tmp_seq = scaler_tmp.transform(np.expand_dims(tmp_seq, 0))
  vbr_seq = scaler_vbr.transform(np.expand_dims(vbr_seq, 0))
  spd_seq = scaler_spd.transform(np.expand_dims(spd_seq, 0))
  data = np.dstack((tmp_seq, vbr_seq, spd_seq))
  res = model.predict(data)
  if res[0, 0] > 0.5:
    print("Failure has been detected")


1/1 [==============================] - 0s 367ms/step
Failure has been detected
1/1 [==============================] - 0s 23ms/step
Failure has been detected
1/1 [==============================] - 0s 21ms/step
Failure has been detected
1/1 [==============================] - 0s 21ms/step
Failure has been detected
1/1 [==============================] - 0s 23ms/step
Failure has been detected
1/1 [==============================] - 0s 21ms/step
